In [ ]:
# | default_exp dataset

# Protein Dataset

> Protein Dataset

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import re
from typing import Tuple, Callable, TypedDict, List

import torch
from torch.utils.data import Dataset

In [ ]:
#| export
def extract_property_tag(name: str) -> Callable:
    def inner(sequence):
        pattern = f"{name}=(.+?) "
        match = re.search(pattern, sequence)
        if match:
            return match.group(1)
        else:
            return None
    return inner

In [ ]:
#| export
class ProteinSequence(TypedDict):
    id: str
    seq: str
    desc: str

In [ ]:
#| export
class ProteinDataset(Dataset):
    def __init__(self, data: List[ProteinSequence], tokenizer: Callable, tag_extractor: Callable):
        xs = []
        ys = []
        
        for item in data:
            xs.append(tag_extractor(item["desc"]))
            ys.append(item["seq"])
        
        encoded_ys = tokenizer.encode_batch(ys)
        encoded_ys = [torch.tensor(e.ids) for e in encoded_ys]
        
        self.xs = xs
        self.ys = encoded_ys

    def __len__(self) -> int:
        return len(self.xs)

    def __getitem__(self, idx: int) -> Tuple[str, str]:
        return self.xs[idx], self.ys[idx]